In [31]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import json
import nltk
import pymorphy2
import string
import pickle
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Тимур\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
with open('../profiles/profiles.json', 'r', encoding='utf-8') as file:
    profiles = json.load(file)
profiles[0]

{'city': 'Москва',
 'age': 24,
 'gender': 'Мужчина',
 'preferences': 'Женщины',
 'name': 'Алексей',
 'description': 'Веселый и общительный, люблю путешествовать и открывать новые места. Занимаюсь фотографией, увлекаюсь историей. Ищу девушку с похожими интересами.'}

In [ ]:
descriptions: list[str] = [profile['description'] for profile in profiles]
descriptions[0]

'Веселый и общительный, люблю путешествовать и открывать новые места. Занимаюсь фотографией, увлекаюсь историей. Ищу девушку с похожими интересами.'

In [15]:
morph = pymorphy2.MorphAnalyzer()
rus_stopwords = set(stopwords.words("russian") + ['это', 'который', 'также', 'будут'])


def clean_text(text: str, stopwords_set: set, morph=morph):
    chars = string.punctuation + '«»-—'
    table = str.maketrans("", "", chars)
    new_text = text.replace('\n', ' ').lower()
    new_text = new_text.translate(table)
    cleaned_text = []
    for word in new_text.split():
        lemma = morph.normal_forms(word)[0]
        if not stopwords_set.intersection(set([lemma])):
            cleaned_text.append(lemma)
    return ' '.join(cleaned_text)

In [16]:
descriptions = [clean_text(description, rus_stopwords) for description in descriptions]
descriptions[0]

'весёлый общительный любить путешествовать открывать новый место заниматься фотография увлекаться история искать девушка похожий интерес'

In [18]:
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower(), language='russian'),tags=[str(i)]) for i, doc in enumerate(descriptions)]
tagged_data[0]

TaggedDocument(words=['весёлый', 'общительный', 'любить', 'путешествовать', 'открывать', 'новый', 'место', 'заниматься', 'фотография', 'увлекаться', 'история', 'искать', 'девушка', 'похожий', 'интерес'], tags=['0'])

In [19]:
model = Doc2Vec(vector_size=50, min_count=2, epochs=50)
model.build_vocab(tagged_data)

In [20]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.dv.get_normed_vectors()[0]

In [25]:
vectors_profiles = {}
for index, vector in enumerate(model.dv.get_normed_vectors()):
    vectors_profiles[str(vector)] = index

In [30]:
vectors_profiles

{'[-0.01082082 -0.22192223 -0.15831323 -0.09336285 -0.01066453 -0.14411207\n  0.11842497  0.20322643 -0.2807324  -0.09780441  0.1389405  -0.17526937\n  0.12183536  0.08326161 -0.00911814  0.05989444  0.17897041  0.10870851\n -0.36421937 -0.07935002  0.03695926  0.05258851  0.30124122 -0.00285852\n  0.2286945  -0.00883734 -0.12962262  0.03250172 -0.21617994 -0.12692891\n  0.06982663  0.2114342   0.03140892  0.01612138 -0.15096188  0.14835942\n  0.05436275 -0.02929312 -0.03630282  0.07226212  0.20783968  0.00710262\n -0.0881127  -0.12651528  0.18923284  0.11567675 -0.14759634 -0.10206653\n  0.10741989 -0.01009761]': 0,
 '[-0.00847498 -0.20894803 -0.15490249 -0.11518563 -0.01431401 -0.13639264\n  0.13542305  0.22576234 -0.2796774  -0.08968423  0.1369877  -0.17432371\n  0.13652807  0.0796975   0.00158057  0.06568955  0.18318768  0.09702617\n -0.35443902 -0.08510292  0.03494482  0.05040812  0.2974257  -0.01869923\n  0.21165265  0.00908445 -0.1315079   0.03165218 -0.22660348 -0.10925205\n  0

In [32]:
with open('./vectors_profiles.bin', 'bw') as file:
    pickle.dump(vectors_profiles, file)

In [23]:
model.save('./model.bin')